# Homework 3 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [1]:
import random
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import torch.optim as optim
from torchsummary import summary
from matplotlib import pyplot as plt
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __getitem__(self, index):
    seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
    result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
    return seq, result

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

D = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=D, size=DATASET_SIZE)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [3]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, hidden_layer, output_size):
    super().__init__()
    #TODO: initialize your submodules
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.hidden_layer = hidden_layer

    self.encoder = nn.GRU(
        input_size=self.output_size, 
        hidden_size=self.hidden_size,
        num_layers=self.hidden_layer,
        batch_first=True, 
        bias=True)
    
    self.decoder = nn.Sequential(
        nn.Linear(self.hidden_size, self.output_size)
    )

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    res, hidden = self.encoder(x)
    # print(res.shape)
    res = self.decoder(res)
    # print(res.shape)
    # print(1)

    return F.log_softmax(res, dim=2), hidden


  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    self.eval()

    x = [ord(i) - ord('a') + 1 for i in s]
    x = torch.FloatTensor(x)
    x = idx_to_onehot(x).unsqueeze(0).to(device)

    # print(x)
    # print(x.unsqueeze(0).shape)
    res, _ = self(x)
    
    res = torch.squeeze(res)
    res = torch.max(res, 1)
    # print(res)
    ans = [chr(ord('a') + i - 1) for i in res.indices]
    # print(ans)
    for idx, (a, b) in enumerate(zip(res.indices, ans)):
      if a == 0:
        ans[idx] = ' '
    
    return ''.join(ans)

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [4]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [5]:
import time
start_time = time.time()

# TODO
def get_loss_and_correct(model, batch, criterion, device, hidden):
  # Implement forward pass and loss calculation for one batch.
  # Remember to move the batch to device.
  # 
  # Return a tuple:
  # - loss for the batch (Tensor)
  # - number of correctly classified examples in the batch (Tensor)
  data, target = batch
  target = target.to(device)
  # print(data.shape)
  data = idx_to_onehot(data).to(device)
  # print(data.shape)
  output, hidden = model(data)
  output = output.permute(0, 2, 1)
  loss = criterion(output, target)

  accurate_count = 0
  # print(accurate_count)

  return loss, accurate_count, hidden


def step(loss, optimizer):
  # Implement backward pass and update.
  # TODO
  loss.backward()
  optimizer.step()


N_EPOCHS = 2
BATCH_SIZE = 64
N = 26
HIDDEN_SIZE=64
HIDDEN_LAYER_SIZE = 4

train_dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, num_workers=4)

model = GRUMemory(HIDDEN_SIZE, HIDDEN_LAYER_SIZE, N+1).to(device)

# criterion = nn.CrossEntropyLoss()
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=1e-3)

model.train()

train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []


for i in range(N_EPOCHS):
  print(i)

  model.train()
  for data, target in train_dataloader:
    data = idx_to_onehot(data).to(device)
    target = target.to(device)

    optimizer.zero_grad()
    out, hidden = model(data)
    out = out.permute(0, 2, 1)
    loss = criterion(out, target)
    loss.backward()

    optimizer.step()


end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

0
1
tests passed


In [6]:
accuracy

1.0

In [7]:
duration

265.5319788455963

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [8]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result
  


### Model

And the model.

In [15]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, hidden_layer, output_size, max_delay):
    super().__init__()
    #TODO
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.hidden_layer = hidden_layer
    self.max_delay = max_delay

    self.encoder = nn.GRU(
        input_size=self.output_size + 1, 
        hidden_size=self.hidden_size,
        num_layers=self.hidden_layer,
        batch_first=True, 
        bias=True)
    
    self.decoder = nn.Sequential(
        nn.Linear(self.hidden_size, self.output_size)
    )

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # map to 3d representation
    delays = delays.view(-1, 1).repeat(1, x.shape[1]).unsqueeze(2)
    # print(delays.shape)
    inp = torch.cat([x, delays], dim=2)
    inp.to(device)

    res, hid = self.encoder(inp)
    # print(res.shape)

    res = self.decoder(res)
    # print(res.shape)
    
    return F.log_softmax(res, dim=2), hidden

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    self.eval()

    x = [ord(i) - ord('a') + 1 for i in s]
    x = torch.FloatTensor(x)
    x = idx_to_onehot(x).unsqueeze(0).to(device)

    delays = torch.Tensor([delay]).to(device)
    res, _ = self(x, delays)
    
    res = torch.squeeze(res)
    res = torch.max(res, 1)
    # print(res)
    ans = [chr(ord('a') + i - 1) for i in res.indices]
    # print(ans)
    for idx, (a, b) in enumerate(zip(res.indices, ans)):
      if a == 0:
        ans[idx] = ' '
    
    return ''.join(ans)


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [10]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    # print(result, s)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [11]:
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
N_EPOCHS = 8
BATCH_SIZE = 256
N = 26
HIDDEN_SIZE=64
HIDDEN_LAYER_SIZE = 2
DATASET_SIZE = 200000

ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=DATASET_SIZE)
train_dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, num_workers=4)

model = VariableDelayGRUMemory(HIDDEN_SIZE, HIDDEN_LAYER_SIZE, N+1, MAX_DELAY).to(device)

# criterion = nn.CrossEntropyLoss()
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=1e-3)

model.train()

for i in range(N_EPOCHS):
  print(i)
  model.train()
  for data, delay, target in train_dataloader:
    data = idx_to_onehot(data).to(device)
    target = target.to(device)
    delay = delay.to(device)
    # print(data.shape)

    optimizer.zero_grad()
    out, hidden = model(data, delay)
    out = out.permute(0, 2, 1)
    loss = criterion(out, target)
    loss.backward()
    optimizer.step()

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

0
1
2
3
4
5
6
7
tests passed


In [12]:
end_time - start_time

292.61908435821533

In [13]:
test_variable_delay_model(model)

0.9997483010319658

In [14]:
!nvidia-smi

Tue Oct 26 11:58:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    34W / 250W |   1055MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
``